In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re
from urllib.parse import urljoin

# -------------------------------
# Setup
# -------------------------------
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

BASE_URL = "https://books.toscrape.com/"
MAX_BOOKS = 1000

RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

# -------------------------------
# Get all categories
# -------------------------------
def get_categories():
    soup = BeautifulSoup(requests.get(BASE_URL).text, "html.parser")
    categories = {}

    for cat in soup.select(".side_categories ul li ul li a"):
        categories[cat.text.strip()] = urljoin(BASE_URL, cat["href"])

    return categories

# -------------------------------
# Scrape books + product description
# -------------------------------
def scrape_books():
    categories = get_categories()
    data = []
    count = 0

    for category, cat_url in categories.items():
        url = cat_url

        while url and count < MAX_BOOKS:
            soup = BeautifulSoup(requests.get(url).text, "html.parser")
            books = soup.select(".product_pod")

            for book in books:
                if count >= MAX_BOOKS:
                    break

                title = book.h3.a["title"]

                # Price (safe)
                price_text = book.select_one(".price_color").text
                price = float(re.sub(r"[^\d.]", "", price_text))

                rating_text = book.p["class"][1]
                rating = RATING_MAP.get(rating_text, 0)

                # ✔ CORRECT PRODUCT PAGE URL
                product_url = urljoin(url, book.h3.a["href"])
                product_page = BeautifulSoup(
                    requests.get(product_url).text, "html.parser"
                )

                # ✔ GUARANTEED DESCRIPTION SCRAPING
                desc = product_page.select_one("#product_description ~ p")
                description = desc.text.strip() if desc else "No description"

                data.append({
                    "title": title,
                    "category": category,
                    "price": price,
                    "rating": rating,
                    "description": description
                })

                count += 1

            next_page = soup.select_one("li.next a")
            url = urljoin(url, next_page["href"]) if next_page else None

    return pd.DataFrame(data)

# -------------------------------
# Sentiment Analysis (SIMPLE)
# -------------------------------
def get_sentiment(text):
    score = sia.polarity_scores(text)["compound"]
    return 1 if score > 0 else 0

# -------------------------------
# Price Adjustment Logic
# -------------------------------
def adjust_price(row):
    price = row["price"]
    category = row["category"].lower()
    sentiment = row["sentiment"]

    # Crime → original price
    if category == "crime":
        return price

    # Negative description → no discount
    if sentiment == 0:
        return price

    # Children → 70% discount
    if category == "childrens":
        return round(price * 0.30, 2)

    # First day opening → 50% discount
    return round(price * 0.50, 2)

# -------------------------------
# MAIN PIPELINE
# -------------------------------
print("Scraping books (with descriptions)...")
df = scrape_books()

print("Running sentiment analysis...")
df["sentiment"] = df["description"].apply(get_sentiment)

print("Applying discounts...")
df["adjusted_price"] = df.apply(adjust_price, axis=1)

df.to_csv("books_sentiment_discount_FINAL.csv", index=False)

print("✅ SUCCESS! CSV created: books_sentiment_discount_FINAL.csv")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Scraping books (with descriptions)...
Running sentiment analysis...
Applying discounts...
✅ SUCCESS! CSV created: books_sentiment_discount_FINAL.csv


Scrapes book data (title, price, rating) from BooksToScrape

Scrapes latest BBC News articles

Extracts important keywords from news using TF-IDF

Checks whether news keywords appear in book titles

If matched → 50% discount applied

Prints results and saves CSV

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# -----------------------------
# NLTK SETUP
# -----------------------------
nltk.download("stopwords")
from nltk.corpus import stopwords
STOP_WORDS = stopwords.words("english")

HEADERS = {"User-Agent": "Mozilla/5.0"}
BOOKS_URL = "https://books.toscrape.com/"
BBC_NEWS_URL = "https://www.bbc.com/news"

RATING_MAP = {
    "One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5
}

# =====================================================
# STEP 1: SCRAPE BOOKS (NO STOCK)
# =====================================================
def scrape_all_books(max_books=1000):
    books = []
    page_url = BOOKS_URL

    while page_url and len(books) < max_books:
        soup = BeautifulSoup(
            requests.get(page_url, headers=HEADERS).text,
            "html.parser"
        )

        for article in soup.select("article.product_pod"):
            if len(books) >= max_books:
                break

            title = article.h3.a["title"]

            price = float(
                re.findall(r"\d+\.\d+", article.select_one(".price_color").text)[0]
            )

            rating_class = article.select_one("p.star-rating")["class"][1]
            rating = RATING_MAP.get(rating_class, 0)

            books.append({
                "title": title,
                "price": price,
                "rating": rating
            })

        next_btn = soup.select_one("li.next a")
        page_url = BOOKS_URL + next_btn["href"] if next_btn else None

    return books


# =====================================================
# STEP 2: SCRAPE TOP 30 BBC NEWS (URL + DESCRIPTION)
# =====================================================
def scrape_bbc_news(max_news=30):
    soup = BeautifulSoup(
        requests.get(BBC_NEWS_URL, headers=HEADERS).text,
        "html.parser"
    )

    urls = []
    for a in soup.select("a[href^='/news']"):
        url = "https://www.bbc.com" + a["href"]
        if url not in urls:
            urls.append(url)
        if len(urls) >= max_news:
            break

    news_data = []
    for url in urls:
        try:
            page = BeautifulSoup(
                requests.get(url, headers=HEADERS, timeout=5).text,
                "html.parser"
            )

            paragraphs = [p.text.strip() for p in page.select("article p")]
            description = " ".join(paragraphs)

            if len(description) < 50:
                continue

            news_data.append({
                "url": url,
                "description": description
            })
        except:
            continue

    return news_data


# =====================================================
# STEP 3: EXTRACT TOP KEYWORDS FROM NEWS
# =====================================================
def extract_top_keywords(news, top_n=20):
    combined_text = " ".join(n["description"] for n in news)
    combined_text = re.sub(r"[^a-zA-Z ]", "", combined_text).lower()

    vectorizer = TfidfVectorizer(
        stop_words=STOP_WORDS,
        max_features=top_n
    )
    vectorizer.fit([combined_text])

    return vectorizer.get_feature_names_out().tolist()


# =====================================================
# STEP 4: APPLY 50% DISCOUNT (MATCH ON TITLE)
# =====================================================
def apply_discount(books, keywords):
    results = []

    for book in books:
        matched = any(k in book["title"].lower() for k in keywords)

        discounted_price = (
            round(book["price"] * 0.50, 2) if matched else book["price"]
        )

        results.append({
            "Title": book["title"],
            "Rating": book["rating"],
            "Original Price": book["price"],
            "Discounted Price": discounted_price,
            "Keyword Matched": matched
        })

    return results


# =====================================================
# STEP 5: PRINT OUTPUT (NEWS DESCRIPTION INCLUDED)
# =====================================================
def print_output(news, keywords, books):
    print("\n================ TOP 30 NEWS =================\n")
    for i, n in enumerate(news, 1):
        matched_keywords = [k for k in keywords if k in n["description"].lower()]

        print(f"News {i}")
        print("URL :", n["url"])
        print("Keywords :", ", ".join(matched_keywords) if matched_keywords else "None")
        print("Description :", n["description"][:400], "...")
        print("-" * 60)

    print("\n================ BOOK RESULTS =================\n")
    for b in books[:20]:
        print("Book Title        :", b["Title"])
        print("Rating            :", b["Rating"])
        print("Original Price    :", b["Original Price"])
        print("Discounted Price  :", b["Discounted Price"])
        print("Keyword Matched   :", "YES" if b["Keyword Matched"] else "NO")
        print("-" * 60)


# =====================================================
# STEP 6: SAVE CSV
# =====================================================
def save_to_csv(books):
    pd.DataFrame(books).to_csv(
        "book_discount_based_on_news.csv",
        index=False
    )
    print("\nCSV saved as: book_discount_based_on_news.csv\n")


# =====================================================
# MAIN
# =====================================================
if __name__ == "__main__":
    print("Scraping books...")
    books = scrape_all_books()

    print("Scraping news...")
    news = scrape_bbc_news()

    print("Extracting keywords...")
    keywords = extract_top_keywords(news)

    print("Applying discount...")
    discounted_books = apply_discount(books, keywords)

    print_output(news, keywords, discounted_books)
    save_to_csv(discounted_books)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Scraping books...
Scraping news...
Extracting keywords...
Applying discount...

================ TOP 30 NEWS =================

News 1
URL : https://www.bbc.com/news
Keywords : bbc, greenland, maduro, media, new, news, one, people, president, russia, say, says, trump, us, venezuela, year, years
Description : It comes after Nicolás Maduro and his wife were seized from Caracas and flown to the US. They have since appeared at a New York court. Venezuelans describe their fear of speaking out as 14 media workers are detained and police check people's social media. Founded 20 years ago, Tomorrowland has grown into one of the most iconic celebrations of electronic music. The talks are held as Russia continue ...
------------------------------------------------------------
News 2
URL : https://www.bbc.com/news/topics/c2vdnvdg6xxt
Keywords : bbc, last, new, one, president, say, trump, us, year, years
Description : Israel's move is criticised by 10 countries, which say the new registration rules

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import time

# -----------------------------
# CONFIG
# -----------------------------
BBC_NEWS_URL = "https://www.bbc.com/news"
MAX_NEWS = 20
OUTPUT_CSV = "bbc_news_sentiment.csv"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# -----------------------------
# GET NEWS LINKS (BBC)
# -----------------------------
def get_news_links(max_results=20):
    r = requests.get(BBC_NEWS_URL, headers=HEADERS)
    soup = BeautifulSoup(r.text, "html.parser")

    links = []
    for a in soup.select("a[href^='/news']"):
        url = "https://www.bbc.com" + a.get("href")
        if url not in links:
            links.append(url)
        if len(links) >= max_results:
            break
    return links

# -----------------------------
# SCRAPE ARTICLE CONTENT
# -----------------------------
def fetch_article_text(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        return " ".join(p.get_text() for p in soup.find_all("p")).strip()
    except:
        return ""

# -----------------------------
# DESCRIPTION EXTRACTION (SAFE)
# -----------------------------
def extract_description(text, max_sentences=3):
    sentences = text.split(". ")
    return ". ".join(sentences[:max_sentences])

# -----------------------------
# KEYWORD EXTRACTION (TF-IDF – SAFE)
# -----------------------------
def extract_keywords(text, max_keywords=10):
    if len(text) < 100:
        return []

    vectorizer = TfidfVectorizer(
        stop_words="english",
        max_features=max_keywords
    )
    tfidf = vectorizer.fit_transform([text])
    return vectorizer.get_feature_names_out().tolist()

# -----------------------------
# MAIN PIPELINE
# -----------------------------
news_links = get_news_links(MAX_NEWS)
results = []

print("\n📰 Top BBC News URLs:")
for i, link in enumerate(news_links, start=1):
    print(f"{i}. {link}")

for url in tqdm(news_links, desc="Processing BBC News"):
    article_text = fetch_article_text(url)

    if len(article_text) < 300:
        continue

    description = extract_description(article_text)
    keywords = extract_keywords(article_text)

    print("\n📰 News URL:", url)
    print("📝 Description:", description)
    print("🔑 Keywords:")

    for kw in keywords:
        print(f"   - {kw}")

        results.append({
            "news_url": url,
            "description": description,
            "keyword": kw
        })

    time.sleep(1)

# -----------------------------
# SAVE CSV
# -----------------------------
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)

print("\n✅ CSV created successfully:", OUTPUT_CSV)



📰 Top BBC News URLs:
1. https://www.bbc.com/news
2. https://www.bbc.com/news/topics/c2vdnvdg6xxt
3. https://www.bbc.com/news/war-in-ukraine
4. https://www.bbc.com/news/us-canada
5. https://www.bbc.com/news/uk
6. https://www.bbc.com/news/world/africa
7. https://www.bbc.com/news/world/asia
8. https://www.bbc.com/news/world/australia
9. https://www.bbc.com/news/world/europe
10. https://www.bbc.com/news/world/latin_america
11. https://www.bbc.com/news/world/middle_east
12. https://www.bbc.com/news/in_pictures
13. https://www.bbc.com/news/bbcindepth
14. https://www.bbc.com/news/bbcverify
15. https://www.bbc.com/news/politics
16. https://www.bbc.com/news/england
17. https://www.bbc.com/news/northern_ireland
18. https://www.bbc.com/news/northern_ireland/northern_ireland_politics
19. https://www.bbc.com/news/scotland
20. https://www.bbc.com/news/scotland/scotland_politics


Processing BBC News:   0%|          | 0/20 [00:00<?, ?it/s]


📰 News URL: https://www.bbc.com/news
📝 Description: It comes after Nicolás Maduro and his wife were seized from Caracas and flown to the US. They have since appeared at a New York court. Venezuelans describe their fear of speaking out as 14 media workers are detained and police check people's social media
🔑 Keywords:
   - bbc
   - killed
   - liam
   - maduro
   - new
   - people
   - rosenior
   - says
   - trump
   - year


Processing BBC News:   5%|▌         | 1/20 [00:01<00:22,  1.20s/it]


📰 News URL: https://www.bbc.com/news/topics/c2vdnvdg6xxt
📝 Description: Israel's move is criticised by 10 countries, which say the new registration rules for international NGOs are "restrictive" and "unacceptable". The US president also warns Hamas will have "hell to pay" if it does not disarm,  after talks with Netanyahu. The BBC revisits one-year-old Siwar Ashour who was temporarily evacuated to Jordan for medical treatment during the Gaza war
🔑 Keywords:
   - bbc
   - gaza
   - hamas
   - international
   - israel
   - police
   - ran
   - say
   - treatment
   - war


...


📰 News URL: https://www.bbc.com/news/scotland
📝 Description: Hundreds of schools remained shut across Scotland on Tuesday, and closures will continue on Wednesday. Temperatures dropped to minus 12.5C (9.5F) in Norfolk overnight - while heavy snow has also hit mainland Europe. The 41-year-old, who was under the influence of cocaine when he crashed in Easter Ross, admitted causing death by dangerous driving
🔑 Keywords:
   - bbc
   - dumfries
   - manager
   - news
   - schools
   - scotland
   - shut
   - snow
   - stories
   - tuesday


Processing BBC News:  95%|█████████▌| 19/20 [00:23<00:01,  1.13s/it]


📰 News URL: https://www.bbc.com/news/scotland/scotland_politics
📝 Description: John Swinney, Anas Sarwar and Russell Findlay have made their pitches for May's Holyrood vote.  Scotland will face Haiti in Boston on Sunday 14 June before taking on Morocco and Brazil in the US. Scotland's resident doctors have voted to strike in January, but the health secretary says it is an outcome both sides want to avoid
🔑 Keywords:
   - bbc
   - external
   - health
   - john
   - parliament
   - public
   - says
   - scotland
   - scottish
   - week


Processing BBC News: 100%|██████████| 20/20 [00:24<00:00,  1.24s/it]


✅ CSV created successfully: bbc_news_sentiment.csv


Scrapes BBC News

Extracts top keywords from real news articles

Scrapes books from books.toscrape.com

Fetches real author, popularity, review using Google Books API

Applies discount logic based on news relevance

Identifies FAST SALE books

Saves everything into a CSV file

Prints a formatted table output

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
from collections import Counter

# --- CONFIGURATION ---
BASE_URL = "https://books.toscrape.com/catalogue/"
PAGE_URL = "https://books.toscrape.com/catalogue/page-{}.html"
BBC_NEWS_URL = "https://www.bbc.com/news"
OUTPUT_FILE = "final_book_report.csv"
TOTAL_PAGES = 3

def get_bbc_keywords():
    """Extracts top keywords from 30 separate BBC News articles."""
    print("Step 1: Extracting top keywords from 30 BBC News articles...")
    try:
        response = requests.get(BBC_NEWS_URL, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            if "/news/" in href and not href.endswith("/news"):
                full_url = "https://www.bbc.com" + href if href.startswith('/') else href
                if full_url not in links: links.append(full_url)
            if len(links) >= 30: break

        all_text = ""
        for link in links:
            try:
                res = requests.get(link, timeout=5)
                article_soup = BeautifulSoup(res.text, 'html.parser')
                all_text += " " + article_soup.get_text().lower()
            except: continue

        words = re.findall(r'\b[a-z]{5,}\b', all_text)
        stopwords = {'their', 'would', 'about', 'there', 'people', 'which', 'could', 'after', 'first', 'government'}
        filtered = [w for w in words if w not in stopwords]
        return [word for word, count in Counter(filtered).most_common(30)]
    except Exception as e:
        print(f"BBC Keyword Error: {e}")
        return []

def get_real_book_metadata(title):
    """Fetches real Authors, Popularity, and Online Reviews via Google Books API."""
    clean_title = re.sub(r'[^\w\s]', '', title).strip()
    search_variants = [clean_title, clean_title.split(':')[0], " ".join(clean_title.split()[:3])]

    for query in search_variants:
        if not query: continue
        try:
            api_url = f"https://www.googleapis.com/books/v1/volumes?q={query.replace(' ', '+')}&maxResults=1"
            response = requests.get(api_url, timeout=5)
            if response.status_code == 200:
                data = response.json()
                if "items" in data:
                    vol = data["items"][0]["volumeInfo"]
                    author = ", ".join(vol.get("authors", []))
                    if author:
                        return {
                            "author": author,
                            "popularity": float(vol.get("averageRating", 4.0)), # Convert to float
                            "review": vol.get("description", "Positive reader feedback.")[:80].replace('\n', ' ') + "..."
                        }
        except: continue
    return {"author": "Unknown Author", "popularity": 0.0, "review": "No reviews found."}

def scrape_and_process():
    news_keywords = get_bbc_keywords()

    with open(OUTPUT_FILE, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # Added 'FAST SALE' column
        writer.writerow(["Title", "Author", "Pop", "Original", "Final", "Status", "Rating", "Stock", "Fast Sale", "Review"])

        for page_num in range(1, TOTAL_PAGES + 1):
            print(f"Scraping Page {page_num}...")
            res = requests.get(PAGE_URL.format(page_num))
            soup = BeautifulSoup(res.text, "html.parser")

            for pod in soup.find_all("article", class_="product_pod"):
                title = pod.h3.a["title"]
                real_info = get_real_book_metadata(title)

                detail_url = BASE_URL + pod.h3.a["href"].replace("../../../", "")
                d_soup = BeautifulSoup(requests.get(detail_url).text, "html.parser")
                desc = d_soup.select_one('article.product_page > p').text.lower() if d_soup.select_one('article.product_page > p') else ""

                price_orig = float(re.sub(r'[^\d.]', '', pod.find("p", class_="price_color").text))
                stock_count = int("".join(filter(str.isdigit, d_soup.find("p", class_="instock availability").text)) or "0")

                # Keyword Discount Logic
                match = any(word in desc for word in news_keywords)
                final_price = round(price_orig * 0.5, 2) if match else price_orig
                status = "50% OFF" if match else "SAME"

                star_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
                num_rating = star_map.get(pod.p['class'][1], 0)

                # --- FAST SALE LOGIC ---
                # A book is a Fast Sale if:
                # 1. It has a high Google Popularity (>= 4.0)
                # 2. AND it has a high local rating (>= 4 stars)
                # 3. AND it is either discounted OR low in stock (< 5)
                is_fast_sale = (
                    real_info['popularity'] >= 4.0 and
                    num_rating >= 4 and
                    (status == "50% OFF" or stock_count < 5)
                )
                fast_sale_status = "YES" if is_fast_sale else "NO"

                writer.writerow([
                    title[:35], real_info['author'][:20], real_info['popularity'],
                    price_orig, final_price, status, num_rating, stock_count, fast_sale_status, real_info['review']
                ])
                time.sleep(0.1)

    # --- TABLE OUTPUT ---
    print("\n" + "="*160)
    header = f"{'TITLE':<35} | {'AUTHOR':<20} | {'POP':<4} | {'ORIG':<7} | {'FINAL':<7} | {'STATUS':<10} | {'STARS':<5} | {'STOCK':<5} | {'FAST SALE':<10} | {'ONLINE REVIEW'}"
    print(header)
    print("-" * 160)

    with open(OUTPUT_FILE, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            print(f"{row[0]:<35} | {row[1]:<20} | {row[2]:<4} | {row[3]:<7} | {row[4]:<7} | {row[5]:<10} | {row[6]:<5} | {row[7]:<5} | {row[8]:<10} | {row[9]}")
    print("="*160)

if __name__ == "__main__":
    scrape_and_process()

Step 1: Extracting top keywords from 30 BBC News articles...
Scraping Page 1...
Scraping Page 2...
Scraping Page 3...

TITLE                               | AUTHOR               | POP  | ORIG    | FINAL   | STATUS     | STARS | STOCK | FAST SALE  | ONLINE REVIEW
----------------------------------------------------------------------------------------------------------------------------------------------------------------
A Light in the Attic                | Shel Silverstein     | 4.0  | 51.77   | 25.89   | 50% OFF    | 3     | 22    | NO         | NOW AVAILABLE AS AN EBOOK! From New York Times bestselling author Shel Silverste...
Tipping the Velvet                  | Sarah Waters         | 4.0  | 53.74   | 26.87   | 50% OFF    | 1     | 20    | NO         | 'Tipping the Velvet' is a wonderfully lush, sensous and bawdy novel set in the m...
Soumission                          | Canada               | 4.0  | 50.1    | 50.1    | SAME       | 1     | 20    | NO         | Positive reader fe